<a href="https://colab.research.google.com/github/sfmajors373/FourthBrainBreastCancer/blob/seg_UNet_update/V2_FEATURE_EXTRACTOR_train_model_inception_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This version use a different data loading method to load tiles 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
import os
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import random
import h5py
import math
import pickle
from datetime import datetime
from numba import cuda 

%matplotlib inline

In [ ]:
# Code snippet nvidia gpu
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
HDF5_FOLDER = '/content/drive/MyDrive/Colab Notebooks/CancerMap/train'
green_layer_only = False
color_normalization_json="CAMELYON16_color_normalization.json"

In [ ]:
# Mannual Data Loading Methods
import h5py
import os

cancer_image = []
cancer_label = []
cancer_mask = []

cancer_image_file = []
cancer_mask_file = []


file = os.listdir(HDF5_FOLDER)
for f_index in range(0, len(file)):
  if file[f_index].find('.hdf5') != -1:
    filepath = HDF5_FOLDER+'/'+file[f_index]

    f = h5py.File(filepath, 'r')
    f_name = list(f.keys())
    dset = np.array(f[f_name[0]])
    dset=dset[:,0:256, 0:256, :]
    print(dset.shape)
    if file[f_index].find('mask') != -1:
      if not len(cancer_mask):
        cancer_mask = dset
      else:
        cancer_mask = np.concatenate((cancer_mask, dset))
      cancer_mask_file.append(f_name)
    elif file[f_index].find('normal') != -1:
      # cancer_label.append(0)
      # cancer_image.append(dset)
      labels = [0]*len(dset)
      cancer_image_file.append(f_name)

      if f_index == 0:
        cancer_image = dset
        cancer_label = labels
      else:
        cancer_image= np.concatenate((cancer_image, dset), axis = 0)
        cancer_label = np.concatenate((cancer_label, labels), axis = 0)
    else:
      labels = [1]*len(dset)
      cancer_image_file.append(f_name)
      if f_index == 0:
        cancer_image = dset
        cancer_label = labels
      else:
        cancer_image= np.concatenate((cancer_image, dset), axis = 0)
        cancer_label = np.concatenate((cancer_label, labels), axis = 0)

# print(cancer_image)
# print(cancer_label)
# print(cancer_mask)

In [ ]:
print(len(cancer_image))
print(len(cancer_label))
print(len(cancer_mask))

In [ ]:
plt.imshow(cancer_image[1])

In [ ]:
plt.imshow(np.squeeze(cancer_mask[1]))

In [ ]:
X_tumor_loc = np.where([cancer_label==1])
len(X_tumor_loc[1])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(cancer_image, cancer_label, test_size = 0.2, random_state = 42)

In [ ]:
print(len(X_train))
print(len(X_test))

In [ ]:
# Plot model loss and accuracy
  
def plot_history(history):
  # This function will plot the model fit process
  print(history.history.keys())
  # summarize history for accuracy
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy')
  plt.ylabel('acc')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()
  # summarize history for loss
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()


In [ ]:
# FEATURE EXTRACTOR MODEL

In [ ]:
model = keras.applications.InceptionV3(input_shape=(256,256,3),
                                        include_top=True,
                                        classes = 1,
                                        classifier_activation="sigmoid",
                                        weights=None)


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.2, batch_size=10, epochs=10)

In [ ]:
plot_history(history)

## Display layer 0 of activation of a normal image

In [ ]:
X_train.shape

In [ ]:
from keras.models import Model
layer_outputs = [layer.output for layer in model.layers]
activation_model = Model(inputs=model.input, outputs=layer_outputs)
image_num = 11600 #normal
activation = activation_model.predict(X_train[image_num].reshape(1,256,256,3))





In [ ]:
y_train[image_num]

In [ ]:
len(activation)

In [ ]:
layer_act = 0

In [ ]:
plt.imshow(activation[layer_act][0, :, :, 0])

In [ ]:
plt.imshow(activation[layer_act][0, :, :, 1])

In [ ]:
plt.imshow(activation[layer_act][0, :, :, 2])

## Display layer 0 of activation of a tumor image

In [ ]:
image_num = 1
activation = activation_model.predict(X_train[image_num].reshape(1,256,256,3))

In [ ]:
y_train[image_num]

In [ ]:
len(activation)

In [ ]:
layer_act = 0

In [ ]:
plt.imshow(activation[layer_act][0, :, :, 0])

In [ ]:
plt.imshow(activation[layer_act][0, :, :, 1])

In [ ]:
plt.imshow(activation[layer_act][0, :, :, 2])

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/CancerMap/model_inceptionv3.hdf5')

In [ ]:
# Test Accuracy
from sklearn.metrics import confusion_matrix, accuracy_score,f1_score,precision_score,recall_score
prediction_values = np.argmax(model.predict(X_test), axis=-1)
print(f'Accuracy = {accuracy_score(y_test, prediction_values)}')
print(f'F1 = {f1_score(y_test, prediction_values,average = None)}')
print(f'Precision = {precision_score(y_test, prediction_values,average=None)}')
print(f'Recall = {recall_score(y_test, prediction_values,average = None)}')
# Display the confusion matrix
print('Confusion matrix =')
print(confusion_matrix(y_test, prediction_values))

In [ ]:
feature_network = Model(model.input, model.get_layer('avg_pool').output)
feature_train = feature_network.predict(X_train)
# feature_test = feature_network.predict(test_data_array)
feature_train_y = y_train

In [ ]:
np.savetxt("/content/drive/MyDrive/Colab Notebooks/CancerMap/feature_extracted.txt", feature_train)
np.savetxt("/content/drive/MyDrive/Colab Notebooks/CancerMap/feature_extracted_y.txt", feature_train_y)

# Select predicted tumor images

In [ ]:
cancer_label

In [ ]:
X_tumor_loc = np.where([cancer_label==1])

In [ ]:
X_tumor=cancer_image[X_tumor_loc[1],]

In [ ]:
X_tumor.shape

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/CancerMap/model_inceptionv3.hdf5')

In [ ]:
from keras.models import Model
layer_outputs = [layer.output for layer in model.layers]
activation_model = Model(inputs=model.input, outputs=layer_outputs)

In [ ]:
tumor = []
imgpath = '/content/drive/MyDrive/Colab Notebooks/CancerMap/tumor_feature/'
for x in range(0,10):
  activation = activation_model.predict(X_tumor[x].reshape(1,256,256,3))
  tumor.append(activation[0])
  imgname = list(cancer_mask_file[x])
  imgname = str(imgname[0])
  for i in range(0,3):
    imgsavepath = imgpath+imgname+'_'+str(i)+'.jpg'
    plt.imsave(imgsavepath, activation[0][0,:,:,i])

In [ ]:
tumor = np.array(tumor)

In [ ]:
tumor.shape

In [ ]:
plt.imshow(np.squeeze(tumor[0][0,:,:,0]))

In [ ]:
tumor= np.squeeze(tumor, axis=(1,))
tumor.shape

In [ ]:
plt.imshow(np.squeeze(tumor[1,:,:,0]))

In [ ]:
plt.imshow(activation[0][0, :, :, 1])

In [ ]:
plt.imshow(np.squeeze(cancer_image[9]))

In [ ]:
plt.imshow(tumor[9][0][0, :, :, 0])

In [ ]:
plt.imshow(np.squeeze(cancer_mask[9]))

# UNet

In [ ]:
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, UpSampling2D
from keras.layers.merge import concatenate

In [ ]:
dim_tiles = 256
depth = 3

In [ ]:

def unet(layer_in = Input(shape=(dim_tiles, dim_tiles, depth))):
    inputs = layer_in
    
    conv1 = Conv2D(64, 3, activation = 'relu',padding = 'same', kernel_initializer = 'he_normal',dilation_rate=2)(inputs) #dilation_rate=2 etc.
    conv1 = Conv2D(64, 3, activation = 'relu',padding = 'same', kernel_initializer = 'he_normal',dilation_rate=2)(conv1) #dilation_rate=2 etc.
    # conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation = 'relu', dilation_rate=2,padding = 'same', kernel_initializer = 'he_normal')(pool1)
    # conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(128, 3, activation = 'relu', dilation_rate=2, padding = 'same', kernel_initializer = 'he_normal')(conv2)
    # conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    # conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    # conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    # conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    # conv4 = BatchNormalization()(conv4)
    drop4 = Dropout(0.5)(conv4)#training=True will enable dropout for test data
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    # conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    # conv5 = BatchNormalization()(conv5)
    drop5 = Dropout(0.5)(conv5) 

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)    

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8) 

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)

    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
    model = Model(inputs = inputs, outputs = conv10)
    return model

In [ ]:
model_unet = unet()

In [ ]:
model_unet.summary()

In [ ]:
model_unet.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001),
                loss='mean_squared_error',
                metrics=['accuracy'])

In [ ]:
print(cancer_mask[0:10].shape)
print(tumor[0:10].shape)

In [ ]:
tumor = np.array(tumor)

In [ ]:
test = np.array(cancer_mask[0:10])
test.shape


In [ ]:
fea_history = model_unet.fit(tumor, test, batch_size=1, epochs= 10)

In [ ]:
layer_outputs_unet = [layer.output for layer in model_unet.layers]
activation_unet = Model(inputs=model_unet.input, outputs=layer_outputs_unet)

In [ ]:
activation_unet_fea = activation_unet.predict(tumor[1].reshape(1,256,256,3))

In [ ]:
np.array(activation_unet_fea[0]).shape

In [ ]:
plt.imshow(activation_unet_fea[0][0, :, :, 1])